<a href="https://colab.research.google.com/github/Theo612/Theo612/blob/main/classif'hackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PARTIE Préparation données**

---



In [1]:
import requests
import pandas as pd

In [2]:
file_path = 'ac.tab'
data = pd.read_csv(file_path, sep='\t', header=None)
data.columns = ['AC', 'Localisation', 'Length']
accession_codes = data['AC']
print(accession_codes)

0        P10719
1        Q38786
2        P15289
3        P25705
4        P00829
          ...  
13000    O60145
13001    O74815
13002    Q3UZD5
13003    P35998
13004    Q07800
Name: AC, Length: 13005, dtype: object


In [3]:
def fetch_protein_data(ac,localisation,longueur):
    url = f"https://www.uniprot.org/uniprot/{ac}.txt"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.text.splitlines()
        protein_data = {}
        protein_data['Accession'] = ac  # Stocker l'Accession Code directement
        sequence_lines=[]
        sequence_flag = False  # Flag pour commencer à lire la séquence si besoin

        for line in data:
            # Rechercher la section de séquence pour récupérer la longueur et le poids moléculaire
            if line.startswith('SQ'):
                sequence_flag = True  # Commencer à lire la séquence
                # SQ line contains useful information about length and weight
                parts = line.split()
                protein_data['Length'] = parts[2]  # Longueur est souvent à la position 2
                protein_data['Poids Moléculaire'] = parts[4]
            if sequence_flag:
                if line.startswith(' '):  # Si la ligne commence par un espace, c'est une ligne de séquence
                    sequence_lines.append(line.strip().replace(' ', ''))  # Ajouter la ligne de séquence
                elif line.startswith('//'):  # Fin de la séquence
                    break  # Arrêter la boucle si on atteint la fin de la séquence

        # Combiner toutes les lignes de séquence en une seule chaîne
        protein_data['Sequence'] = ''.join(sequence_lines)
        protein_data['Localisation'] = localisation
        protein_data['Length peptide']=longueur

        # Retourner les informations collectées
        return protein_data

    else:
        print(f"Erreur pour {ac}: {response.status_code}")
        return None
# Liste d'AC à traiter
ac_list = data['AC']
localisation_list = data['Localisation']
targeting_peptide_list=data['Length']
ac_list_limited = ac_list[:10]
localisation_list_limited=localisation_list[:10]
targeting_peptide_list_limited=targeting_peptide_list[:10]
# Liste pour stocker les données récupérées
protein_data_list = []

# Récupérer les données pour chaque AC
for ac,localisation,longueur in zip(ac_list_limited, localisation_list_limited,targeting_peptide_list_limited):
    protein_data = fetch_protein_data(ac,localisation,longueur)
    if protein_data:
        protein_data_list.append(protein_data)
print(protein_data_list)


[{'Accession': 'P10719', 'Length': '529', 'Poids Moléculaire': '56354', 'Sequence': 'MLSLVGRVASASASGALRGLNPLAALPQAHLLLRTAPAGVHPARDYAAQSSAAPKAGTATGQIVAVIGAVVDVQFDEGLPPILNALEVQGRESRLVLEVAQHLGESTVRTIAMDGTEGLVRGQKVLDSGAPIKIPVGPETLGRIMNVIGEPIDERGPIKTKQFAPIHAEAPEFIEMSVEQEILVTGIKVVDLLAPYAKGGKIGLFGGAGVGKTVLIMELINNVAKAHGGYSVFAGVGERTREGNDLYHEMIESGVINLKDATSKVALVYGQMNEPPGARARVALTGLTVAEYFRDQEGQDVLLFIDNIFRFTQAGSEVSALLGRIPSAVGYQPTLATDMGTMQERITTTKKGSITSVQAIYVPADDLTDPAPATTFAHLDATTVLSRAIAELGIYPAVDPLDSTSRIMDPNIVGSEHYDVARGVQKILQDYKSLQDIIAILGMDELSEEDKLTVSRARKIQRFLSQPFQVAEVFTGHMGKLVPLKETIKGFQQILAGDYDHLPEQAFYMVGPIEEAVAKADKLAEEHGS', 'Localisation': 'MT', 'Length peptide': 46}, {'Accession': 'Q38786', 'Length': '574', 'Poids Moléculaire': '65041', 'Sequence': 'MALLCSALSNSTHPSFRSHIGANSENLWHLSADPAQKSKRRCNLTLSSRAARISSALESAKQVKPWQVPKRDWFPPEFMFGAASAAYQIEGAWNEGGKGPSSWDNFCHSHPDRIMDKSNADVAANSYYMYKEDVRMLKEIGMDSYRFSISWPRILPKGTLDGGINHEGIQYYNDLLDCLIENGIKPYITLFHWDTPQALADEYKDFLDRRIVKDYTDYATVCFEHFGDKVKNWFTFNEPHSFCGLGYGTGLHAP

In [4]:
# Indices d'hydropathie des acides aminés
hydropathy_index = {
    'A': 1.8,  # Alanine
    'R': -4.5,  # Arginine
    'N': -3.5,  # Asparagine
    'D': -3.5,  # Acide Aspartique
    'C': 2.5,  # Cystéine
    'E': -3.5,  # Acide Glutamique
    'Q': -3.5,  # Glutamine
    'G': -0.4,  # Glycine
    'H': -3.2,  # Histidine
    'I': 4.5,  # Isoleucine
    'L': 3.8,  # Leucine
    'K': -3.9,  # Lysine
    'M': 1.9,  # Méthionine
    'F': 2.8,  # Phénylalanine
    'P': -1.6,  # Proline
    'S': -0.8,  # Sérine
    'T': -0.7,  # Thréonine
    'W': -0.9,  # Tryptophane
    'Y': -1.3,  # Tyrosine
    'V': 4.2,  # Valine
}

def calculat_hydropathy(sequence):
    total_hydropathy=0 ; nb=0
    for amino_acid in sequence :
      value = hydropathy_index.get(amino_acid, 0)
      total_hydropathy += value
      nb+=1
    return total_hydropathy / nb

#on rajoute à nos seq
for i in protein_data_list:
    sequence=i['Sequence']
    indice = calculat_hydropathy(sequence)
    print(f"L'indice d'hydropathie pour la séquence est : {indice:.2f}")
    i['hydropathie']=indice
print(protein_data_list)


L'indice d'hydropathie pour la séquence est : 0.03
L'indice d'hydropathie pour la séquence est : -0.44
L'indice d'hydropathie pour la séquence est : 0.01
L'indice d'hydropathie pour la séquence est : -0.07
L'indice d'hydropathie pour la séquence est : 0.04
L'indice d'hydropathie pour la séquence est : -0.54
L'indice d'hydropathie pour la séquence est : 0.02
L'indice d'hydropathie pour la séquence est : 0.04
L'indice d'hydropathie pour la séquence est : 0.04
L'indice d'hydropathie pour la séquence est : -0.66
[{'Accession': 'P10719', 'Length': '529', 'Poids Moléculaire': '56354', 'Sequence': 'MLSLVGRVASASASGALRGLNPLAALPQAHLLLRTAPAGVHPARDYAAQSSAAPKAGTATGQIVAVIGAVVDVQFDEGLPPILNALEVQGRESRLVLEVAQHLGESTVRTIAMDGTEGLVRGQKVLDSGAPIKIPVGPETLGRIMNVIGEPIDERGPIKTKQFAPIHAEAPEFIEMSVEQEILVTGIKVVDLLAPYAKGGKIGLFGGAGVGKTVLIMELINNVAKAHGGYSVFAGVGERTREGNDLYHEMIESGVINLKDATSKVALVYGQMNEPPGARARVALTGLTVAEYFRDQEGQDVLLFIDNIFRFTQAGSEVSALLGRIPSAVGYQPTLATDMGTMQERITTTKKGSITSVQAIYVPADDLTDPAPATTFAHLDATTVLSRAIAELGIYPAVDPL

In [5]:
from collections import Counter

# Fonction pour calculer la composition selon les groupes fonctionnels d'acides aminés
def functional_group_composition(sequence):
    hydrophobe = 'AILMFVW'
    hydrophile = 'STYNQ'
    basique = 'KRH'
    acide = 'DE'

    counts = Counter(sequence)
    total_length = len(sequence)

    composition = {
        'Hydrophobe': sum(counts[aa] for aa in hydrophobe) / total_length,
        'Hydrophile': sum(counts[aa] for aa in hydrophile) / total_length,
        'Basique': sum(counts[aa] for aa in basique) / total_length,
        'Acidie': sum(counts[aa] for aa in acide) / total_length
    }

    return composition

# Exemple d'ajout de la composition des groupes fonctionnels à une protéine
def enrich_protein_data_with_functional_groups(protein_data):
    sequence = protein_data['Sequence']
    composition = functional_group_composition(sequence)
    protein_data.update(composition)
    print(protein_data)
    return protein_data

# Appliquer cette fonction à chaque protéine dans la liste
for protein_data in protein_data_list:
    enrich_protein_data_with_functional_groups(protein_data)


{'Accession': 'P10719', 'Length': '529', 'Poids Moléculaire': '56354', 'Sequence': 'MLSLVGRVASASASGALRGLNPLAALPQAHLLLRTAPAGVHPARDYAAQSSAAPKAGTATGQIVAVIGAVVDVQFDEGLPPILNALEVQGRESRLVLEVAQHLGESTVRTIAMDGTEGLVRGQKVLDSGAPIKIPVGPETLGRIMNVIGEPIDERGPIKTKQFAPIHAEAPEFIEMSVEQEILVTGIKVVDLLAPYAKGGKIGLFGGAGVGKTVLIMELINNVAKAHGGYSVFAGVGERTREGNDLYHEMIESGVINLKDATSKVALVYGQMNEPPGARARVALTGLTVAEYFRDQEGQDVLLFIDNIFRFTQAGSEVSALLGRIPSAVGYQPTLATDMGTMQERITTTKKGSITSVQAIYVPADDLTDPAPATTFAHLDATTVLSRAIAELGIYPAVDPLDSTSRIMDPNIVGSEHYDVARGVQKILQDYKSLQDIIAILGMDELSEEDKLTVSRARKIQRFLSQPFQVAEVFTGHMGKLVPLKETIKGFQQILAGDYDHLPEQAFYMVGPIEEAVAKADKLAEEHGS', 'Localisation': 'MT', 'Length peptide': 46, 'hydropathie': 0.034026465028355234, 'Hydrophobe': 0.41776937618147447, 'Hydrophile': 0.2003780718336484, 'Basique': 0.10964083175803403, 'Acidie': 0.12098298676748583}
{'Accession': 'Q38786', 'Length': '574', 'Poids Moléculaire': '65041', 'Sequence': 'MALLCSALSNSTHPSFRSHIGANSENLWHLSADPAQKSKRRCNLTLSSRAARISSALESAKQVKPWQVPKRDWFPPEFMFGAASAAY

In [6]:
#Après avoir recup les features, on sort les seq
for protein_data in protein_data_list:
    if 'Sequence' in protein_data:
        del protein_data['Sequence']
print(protein_data_list)

[{'Accession': 'P10719', 'Length': '529', 'Poids Moléculaire': '56354', 'Localisation': 'MT', 'Length peptide': 46, 'hydropathie': 0.034026465028355234, 'Hydrophobe': 0.41776937618147447, 'Hydrophile': 0.2003780718336484, 'Basique': 0.10964083175803403, 'Acidie': 0.12098298676748583}, {'Accession': 'Q38786', 'Length': '574', 'Poids Moléculaire': '65041', 'Localisation': 'CH', 'Length peptide': 55, 'hydropathie': -0.4416376306620207, 'Hydrophobe': 0.343205574912892, 'Hydrophile': 0.2421602787456446, 'Basique': 0.13588850174216027, 'Acidie': 0.12195121951219512}, {'Accession': 'P15289', 'Length': '507', 'Poids Moléculaire': '53588', 'Localisation': 'SP', 'Length peptide': 18, 'hydropathie': 0.011637080867849967, 'Hydrophobe': 0.3767258382642998, 'Hydrophile': 0.19723865877712032, 'Basique': 0.09664694280078895, 'Acidie': 0.08875739644970414}, {'Accession': 'P25705', 'Length': '553', 'Poids Moléculaire': '59751', 'Localisation': 'MT', 'Length peptide': 43, 'hydropathie': -0.06672694394213

In [7]:
# Dictionnaire pour convertir le texte en labels numériques
label_mapping = {
    'SP': 0,  # Signal peptide
    'MT': 1,  # Mitochondrial transit peptide
    'CH': 2,  # Chloroplast transit peptide
    'TH': 3,  # Thylakoidal lumen composite transit peptide
    'Other': 4  # No targeting peptide
}

# Exemple de transformation des localisations en labels
for protein_data in protein_data_list:
    if 'Localisation' in protein_data:
        protein_data['Localisation'] = label_mapping[protein_data['Localisation']]
print(protein_data_list)


[{'Accession': 'P10719', 'Length': '529', 'Poids Moléculaire': '56354', 'Localisation': 1, 'Length peptide': 46, 'hydropathie': 0.034026465028355234, 'Hydrophobe': 0.41776937618147447, 'Hydrophile': 0.2003780718336484, 'Basique': 0.10964083175803403, 'Acidie': 0.12098298676748583}, {'Accession': 'Q38786', 'Length': '574', 'Poids Moléculaire': '65041', 'Localisation': 2, 'Length peptide': 55, 'hydropathie': -0.4416376306620207, 'Hydrophobe': 0.343205574912892, 'Hydrophile': 0.2421602787456446, 'Basique': 0.13588850174216027, 'Acidie': 0.12195121951219512}, {'Accession': 'P15289', 'Length': '507', 'Poids Moléculaire': '53588', 'Localisation': 0, 'Length peptide': 18, 'hydropathie': 0.011637080867849967, 'Hydrophobe': 0.3767258382642998, 'Hydrophile': 0.19723865877712032, 'Basique': 0.09664694280078895, 'Acidie': 0.08875739644970414}, {'Accession': 'P25705', 'Length': '553', 'Poids Moléculaire': '59751', 'Localisation': 1, 'Length peptide': 43, 'hydropathie': -0.06672694394213376, 'Hydrop

In [8]:
# Convertir en DataFrame
df = pd.DataFrame(protein_data_list)

# Afficher le DataFrame
print(df)

  Accession Length Poids Moléculaire  Localisation  Length peptide  \
0    P10719    529             56354             1              46   
1    Q38786    574             65041             2              55   
2    P15289    507             53588             0              18   
3    P25705    553             59751             1              43   
4    P00829    528             56284             1              48   
5    P19633    406             46449             0              34   
6    Q96252    203             21548             1              26   
7    Q12165    160             17020             1              22   
8    Q40089    200             21318             1              21   
9    Q03352    105             11281             0              17   

   hydropathie  Hydrophobe  Hydrophile   Basique    Acidie  
0     0.034026    0.417769    0.200378  0.109641  0.120983  
1    -0.441638    0.343206    0.242160  0.135889  0.121951  
2     0.011637    0.376726    0.197239  0.0966

# **PARTIE ML**

In [ ]:
X = data[['Length']]  # Utiliser la longueur et autre comme feature
y = data['Localisation']  # Utiliser la localisation comme label

# Encoder les labels (localisation) si nécessaire
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Maintenant, X et y_encoded peuvent être utilisés pour entraîner le modèle SVM


In [ ]:
from sklearn.model_selection import train_test_split

# Séparer les données en train/test (75% entraînement, 25% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Créer un scaler et ajuster les données d'entraînement
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Ne pas ajuster à nouveau sur le test


In [ ]:
from sklearn.svm import SVC

# Créer un modèle SVM
model = SVC(kernel='linear')  # Utiliser le noyau linéaire (tu peux essayer d'autres noyaux comme 'rbf' ou 'poly')
model.fit(X_train_scaled, y_train)  # Entraîner le modèle


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test_scaled)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Afficher le rapport de classification
print(classification_report(y_test, y_pred))

# Afficher la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)
